<a href="https://colab.research.google.com/github/AmiraAshraf2/NLP/blob/main/NLP_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [ ]:
text_generation_pipeline = pipeline("text-generation")
documentA = text_generation_pipeline("the sun is yellow")[0]["generated_text"]
documentB = text_generation_pipeline("the moon is white")[0]["generated_text"]

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
documentA

'the sun is yellow, but it does not look like any other color in the sky at all! It was once the common belief that blue would be the last color of choice, that is, not blue. It is possible that the blue is due'

In [ ]:
documentB

"the moon is white. The stars are black. The Sun is yellow. It's the Moon."

In [ ]:
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())

    tokens = nltk.word_tokenize(text)


    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    return filtered_tokens

In [ ]:
bagofwordsA = preprocess_text(documentA)
bagofwordsB = preprocess_text(documentB)

In [ ]:
bagofwordsA

['sun',
 'yellow',
 'look',
 'like',
 'color',
 'sky',
 'common',
 'belief',
 'blue',
 'would',
 'last',
 'color',
 'choice',
 'blue',
 'possible',
 'blue',
 'due']

In [ ]:
bagofwordsB

['moon', 'white', 'stars', 'black', 'sun', 'yellow', 'moon']

In [ ]:
uniquewords = set(bagofwordsA).union(set(bagofwordsB))

In [ ]:
uniquewords

{'belief',
 'black',
 'blue',
 'choice',
 'color',
 'common',
 'due',
 'last',
 'like',
 'look',
 'moon',
 'possible',
 'sky',
 'stars',
 'sun',
 'white',
 'would',
 'yellow'}

In [ ]:
numofwordsA = dict.fromkeys (uniquewords, 0)
for word in bagofwordsA:
  numofwordsA [word] += 1

numOfWordsB = dict.fromkeys (uniquewords, 0)

for word in bagofwordsB:
   numOfWordsB [word] += 1

In [ ]:
numofwordsA

{'belief': 1,
 'color': 2,
 'sun': 1,
 'sky': 1,
 'stars': 0,
 'black': 0,
 'yellow': 1,
 'last': 1,
 'possible': 1,
 'look': 1,
 'like': 1,
 'choice': 1,
 'moon': 0,
 'blue': 3,
 'white': 0,
 'due': 1,
 'would': 1,
 'common': 1}

In [ ]:
numOfWordsB

{'belief': 0,
 'color': 0,
 'sun': 1,
 'sky': 0,
 'stars': 1,
 'black': 1,
 'yellow': 1,
 'last': 0,
 'possible': 0,
 'look': 0,
 'like': 0,
 'choice': 0,
 'moon': 2,
 'blue': 0,
 'white': 1,
 'due': 0,
 'would': 0,
 'common': 0}

In [ ]:
def computeTF (wordDict, bagofwords):

  tfDict = {}

  bagofwordsCount = len(bagofwords)

  for word, count in wordDict.items(): tfDict[word] = count / float (bagofwordsCount)
  return tfDict

In [ ]:
tfA = computeTF (numofwordsA, bagofwordsA)

tfB = computeTF (numOfWordsB, bagofwordsB)

In [ ]:
tfA

{'belief': 0.058823529411764705,
 'color': 0.11764705882352941,
 'sun': 0.058823529411764705,
 'sky': 0.058823529411764705,
 'stars': 0.0,
 'black': 0.0,
 'yellow': 0.058823529411764705,
 'last': 0.058823529411764705,
 'possible': 0.058823529411764705,
 'look': 0.058823529411764705,
 'like': 0.058823529411764705,
 'choice': 0.058823529411764705,
 'moon': 0.0,
 'blue': 0.17647058823529413,
 'white': 0.0,
 'due': 0.058823529411764705,
 'would': 0.058823529411764705,
 'common': 0.058823529411764705}

In [ ]:
tfB

{'belief': 0.0,
 'color': 0.0,
 'sun': 0.14285714285714285,
 'sky': 0.0,
 'stars': 0.14285714285714285,
 'black': 0.14285714285714285,
 'yellow': 0.14285714285714285,
 'last': 0.0,
 'possible': 0.0,
 'look': 0.0,
 'like': 0.0,
 'choice': 0.0,
 'moon': 0.2857142857142857,
 'blue': 0.0,
 'white': 0.14285714285714285,
 'due': 0.0,
 'would': 0.0,
 'common': 0.0}

In [ ]:
def computeIDF (documents):

  import math

  N = len(documents)

  idfDict = dict.fromkeys(documents[0].keys(), 0)

  for document in documents:

    for word, val in document.items():

      if val > 0:

        idfDict[word] += 1
  for word, val in idfDict.items():

    idfDict[word] = math.log(N / float(val))

  return idfDict

In [ ]:
idfs = computeIDF([numofwordsA, numOfWordsB])

In [ ]:
idfs

{'belief': 0.6931471805599453,
 'color': 0.6931471805599453,
 'sun': 0.0,
 'sky': 0.6931471805599453,
 'stars': 0.6931471805599453,
 'black': 0.6931471805599453,
 'yellow': 0.0,
 'last': 0.6931471805599453,
 'possible': 0.6931471805599453,
 'look': 0.6931471805599453,
 'like': 0.6931471805599453,
 'choice': 0.6931471805599453,
 'moon': 0.6931471805599453,
 'blue': 0.6931471805599453,
 'white': 0.6931471805599453,
 'due': 0.6931471805599453,
 'would': 0.6931471805599453,
 'common': 0.6931471805599453}

In [ ]:
def computeTFIDF (tfBagOfWords, idfs):

  tfidf = {}

  for word, tf in tfBagOfWords.items():

    tfidf[word] = tf * idfs [word]

  return tfidf

In [ ]:
import pandas as pd

In [ ]:
tfidfA = computeTFIDF(tfA, idfs)

tfidfB = computeTFIDF (tfB, idfs)

df = pd.DataFrame ([tfidfA, tfidfB])

In [ ]:
df

,belief,color,sun,sky,stars,black,yellow,last,possible,look,like,choice,moon,blue,white,due,would,common
0,0.040773,0.081547,0.0,0.040773,0.000000,0.000000,0.0,0.040773,0.040773,0.040773,0.040773,0.040773,0.000000,0.12232,0.000000,0.040773,0.040773,0.040773
1,0.000000,0.000000,0.0,0.000000,0.099021,0.099021,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.198042,0.00000,0.099021,0.000000,0.000000,0.000000


In [ ]:
documents = [documentA, documentB]


In [ ]:
uniquewords

{'belief',
 'black',
 'blue',
 'choice',
 'color',
 'common',
 'due',
 'last',
 'like',
 'look',
 'moon',
 'possible',
 'sky',
 'stars',
 'sun',
 'white',
 'would',
 'yellow'}

In [ ]:
# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(vocabulary=uniquewords)

# Fit the vectorizer and transform the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Get feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Display TF-IDF matrix
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())

# Display feature names
print("\nFeature Names (Words):")
print(feature_names)

TF-IDF Matrix:
[[0.20407106 0.         0.61221318 0.20407106 0.40814212 0.20407106
  0.20407106 0.20407106 0.20407106 0.20407106 0.         0.20407106
  0.20407106 0.         0.14519824 0.         0.20407106 0.14519824]
 [0.         0.35327777 0.         0.         0.         0.
  0.         0.         0.         0.         0.70655553 0.
  0.         0.35327777 0.25136004 0.35327777 0.         0.25136004]]

Feature Names (Words):
['belief' 'black' 'blue' 'choice' 'color' 'common' 'due' 'last' 'like'
 'look' 'moon' 'possible' 'sky' 'stars' 'sun' 'white' 'would' 'yellow']


In [ ]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=['DocumentA', 'DocumentB'])

In [ ]:
tfidf_df

,belief,black,blue,choice,color,common,due,last,like,look,moon,possible,sky,stars,sun,white,would,yellow
DocumentA,0.204071,0.000000,0.612213,0.204071,0.408142,0.204071,0.204071,0.204071,0.204071,0.204071,0.000000,0.204071,0.204071,0.000000,0.145198,0.000000,0.204071,0.145198
DocumentB,0.000000,0.353278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.706556,0.000000,0.000000,0.353278,0.251360,0.353278,0.000000,0.251360


In [ ]:
df

,belief,color,sun,sky,stars,black,yellow,last,possible,look,like,choice,moon,blue,white,due,would,common
0,0.040773,0.081547,0.0,0.040773,0.000000,0.000000,0.0,0.040773,0.040773,0.040773,0.040773,0.040773,0.000000,0.12232,0.000000,0.040773,0.040773,0.040773
1,0.000000,0.000000,0.0,0.000000,0.099021,0.099021,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.198042,0.00000,0.099021,0.000000,0.000000,0.000000
